<a href="https://colab.research.google.com/github/ThijsVercammen/Masterproef/blob/main/Faster_RCNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instaleer nodige tools

In [ ]:
%%shell

pip install cython
pip install torchvision
!pip install "opencv-python-headless<4.3"
!pip install brambox
pip install onnxruntime
wget https://machinelearningmastery.com/wp-content/uploads/2019/03/zebra.jpg

git clone https://github.com/pdollar/coco/
cd coco/PythonAPI
make
python setup.py install
cd ../..
rm -r coco

Importeer de nodige bibliotheken

In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image
import torchvision.transforms.functional as F
from collections import OrderedDict
from typing import Tuple, List, Dict, Optional, Union
import torchvision
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile
from torch import nn, Tensor
import torch.onnx
import onnxruntime as ort
import datetime
from datetime import datetime
import numpy as np

Imorteer het Faster-RCNN model en voer het uit een test afbeelding

In [ ]:
m = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

im = Image.open("/content/zebra.jpg")
res_im = im.resize((224, 224))
convert_tensor = torchvision.transforms.ToTensor()
a = convert_tensor(res_im)
img2 = F.convert_image_dtype(a, torch.float)
img1 = F.convert_image_dtype(a, torch.uint8)

m.eval()

now = datetime.now()
with torch.no_grad():
    prediction = m([img2])
print("Snelheid: {}".format(datetime.now()-now))

boxes = prediction[0]['boxes']

print(prediction)
drawn_boxes = draw_bounding_boxes(img1, boxes, colors="red")
Image.fromarray(drawn_boxes.mul(255).permute(1, 2, 0).byte().numpy())

Splits het Faster-RCNN model op in verschillende componenten en voer elke component apart uit op de testafbeelding

In [ ]:
m.cpu()
transf = m.transform
backbone = m.backbone
rpn = m.rpn
roi_heads = m.roi_heads
#rpn_a = m.rpn.anchor_generator
#rpn_h = m.rpn.head
#roi_heads_1 = m.roi_heads.box_roi_pool
#roi_heads_2 = m.roi_heads.box_head
#roi_heads_3 = m.roi_heads.box_predictor


In [ ]:
img1 = read_image("/content/zebra.jpg")
img = F.convert_image_dtype(img1, torch.float)
original_image_sizes: List[Tuple[int, int]] = []

val = img.shape[-2:]
assert len(val) == 2
original_image_sizes.append((val[0], val[1]))

#transform input
imgages, targets = transf([img])

#extract features
features  = backbone(imgages.tensors)

#region proposals
prop, prop_l = rpn(imgages, features)

#detections
det, det_l = roi_heads(features, prop, imgages.image_sizes, targets)
det_p = transf.postprocess(det, imgages.image_sizes, original_image_sizes)

boxes = det_p[0]['boxes']

print(det_p)
drawn_boxes = draw_bounding_boxes(img1, boxes, colors="red")
Image.fromarray(drawn_boxes.mul(255).permute(1, 2, 0).byte().numpy())

Sla het model op en converteer het naar een model dat kan uitgevoerd worden in android studio

In [ ]:
m = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

torch.save(m, "model.pth")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
example = torch.randn(1, 3, 160,160)
m.eval()
m.to('cpu')

traced_script_module = torch.jit.script(m)
traced_script_module.to('cpu')
traced_script_module.save('./model.pt')

#transf.to('cpu')
#scripted_transf = torch.jit.script(transf)
#scripted_transf.save('./transf.pt')

#backbone.to('cpu')
#traced_backbone = torch.jit.script(backbone)
#traced_backbone.save('./backbone.pt')

#rpn_h.to('cpu')
#scripted_rpn_h = torch.jit.script(rpn_h)
#scripted_rpn.save('./rpn_anchors.pt')

# optimize posible for mobile
#rpn.to('cpu')
#scripted_rpn = torch.jit.script(rpn)
#scripted_rpn.save('./rpn.pt')

#roi_heads_1.to('cpu')
#scripted_roi_heads_1 = torch.jit.script(roi_heads_1)
#scripted_roi_heads_1_opt = optimize_for_mobile(scripted_roi_heads_1)
#scripted_roi_heads_1_opt.save('./roi_h_1.pt')


#roi_heads_2.to('cpu')
#scripted_roi_heads_2 = torch.jit.script(roi_heads_2)
#scripted_roi_heads_2_opt = optimize_for_mobile(scripted_roi_heads_2)
#scripted_roi_heads_2_opt._save_for_lite_interpreter("mod1.ptl")
#scripted_roi_heads_2.save('./roi_h_2.pt')


#roi_heads_3.to('cpu')
#scripted_roi_heads_3 = torch.jit.script(roi_heads_3)
#scripted_roi_heads_3_opt = optimize_for_mobile(scripted_roi_heads_3)
#scripted_roi_heads_3_opt._save_for_lite_interpreter("mod1.ptl")
#scripted_roi_heads_3.save('./roi_h_3.pt')

Voer het geconverteerde model uit op de testafbeelding

In [ ]:
im = Image.open("/content/zebra.jpg")
res_im = im.resize((416, 416))
convert_tensor = transforms.ToTensor()
a = convert_tensor(res_im)

now = datetime.now()
out = traced_script_module([a])
print("Snelheid: {}".format(datetime.now()-now))
out

Converteer het model naar ONNX formaat

In [ ]:
example = torch.randn(1, 3, 416,416)
example.to('cpu')
torch.onnx.export(m,               # model being run
                  example,                         # model input (or a tuple for multiple inputs)
                  "model_py.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=12,         # the ONNX version to export the model to
                  do_constant_folding=True,
                  input_names = ['input_1'],   # the model's input names
                  output_names = ['output'],
                  )

Voer het ONNX model uit met een testafbeelding

In [ ]:
im = Image.open("/content/zebra.jpg")
res_im = im.resize((416, 416))
convert_tensor = transforms.ToTensor()

a = convert_tensor(res_im)
img = F.convert_image_dtype(a, torch.float)
b = img.unsqueeze(0)
input_data = np.asarray(img, dtype=np.float32)


ort_session = ort.InferenceSession("model_py.onnx")

now = datetime.now()
outputs = ort_session.run(
    None,
    {"input_1": [input_data]},
)
print("Snelheid: {}".format(datetime.now()-now))
outputs


Download testdata

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

import torchvision.datasets as dset

path2data="/content/val2017"
path2json="/content/annotations/instances_val2017.json"

coco_train = dset.CocoDetection(root = path2data,
                                annFile = path2json)

loading annotations into memory...
Done (t=0.78s)
creating index...
index created!


Evalueer het standaard, mobiel en ONNX model
(coco.names file met de coco labels moet hiervoor geupload worden)

In [ ]:
from torch.nn.modules.module import T

ort_session = ort.InferenceSession("model_py.onnx")

cc = coco_train.coco
m.eval()
z = 0
class_label_map=[lines.strip() for lines in open('coco.names')]
results_cs = []
results_cs_m = []
results_cs_o = []

for x in cc.imgs:
  val = cc.imgs.get(x)
  imgpath = "/content/val2017/"+val['file_name']
  print("index: {} Image: {}".format(z, imgpath))
  im = Image.open(imgpath)
  resized_scale = 416

  res_im = im.resize((resized_scale, resized_scale))
  convert_tensor = torchvision.transforms.ToTensor()
  tensor = convert_tensor(res_im)
  img1 = F.convert_image_dtype(tensor, torch.float)
  input_data = np.asarray(img1, dtype=np.float32)

  width = val['width']
  height = val['height']

  imageName = val['file_name']
  image_e = imageName[:12]

 
  try:
    with torch.no_grad():
      prediction_1 = m([img1])
      prediction_2 = traced_script_module([img1])


    scores = prediction_1[0]['scores']
    labels = prediction_1[0]['labels']
    boxes = prediction_1[0]['boxes']

    
    for i in range(len(scores)):
      if float(scores[i]) >= 0.5:
        box1 = [float((boxes[i][0]/resized_scale)*width), float((boxes[i][1]/resized_scale)*height), float((boxes[i][2]/resized_scale)*width), float((boxes[i][3]/resized_scale)*height)]
        w1, h1 = box1[2]-box1[0], box1[3]-box1[1]
        results_cs.append([int(image_e), class_label_map[int(labels[i])-1], None, box1[0], box1[1], w1, h1, float(scores[i])])
  except:
    print("error default model {} ".format(val))


  try:
    scores = prediction_2[1][0]['scores']
    labels = prediction_2[1][0]['labels']
    boxes = prediction_2[1][0]['boxes']

    for i in range(len(scores)):
      if float(scores[i]) >= 0.5:
        box1 = [float((boxes[i][0]/resized_scale)*width), float((boxes[i][1]/resized_scale)*height), float((boxes[i][2]/resized_scale)*width), float((boxes[i][3]/resized_scale)*height)]
        w1, h1 = box1[2]-box1[0], box1[3]-box1[1]
        results_cs_m.append([int(image_e), class_label_map[int(labels[i])-1], None, box1[0], box1[1], w1, h1, float(scores[i])])
  except:
    print("error mobiel model {} ".format(val))

  try:
    outputs = ort_session.run(None, {"input_1": [input_data]})
    scores = outputs[2]
    labels = outputs[1]
    boxes = outputs[0]

    for i in range(len(scores)):
      if float(scores[i]) >= 0.5:
        box1 = [float((boxes[i][0]/resized_scale)*width), float((boxes[i][1]/resized_scale)*height), float((boxes[i][2]/resized_scale)*width), float((boxes[i][3]/resized_scale)*height)]
        w1, h1 = box1[2]-box1[0], box1[3]-box1[1]
        results_cs_o.append([int(image_e), class_label_map[int(labels[i])-1], None, box1[0], box1[1], w1, h1, float(scores[i])])
  except:
    print("error onnx model {} ".format(val))

  z += 1


Evalueer resultaat met de brambox bibliotheek

In [ ]:

# Basic imports
import brambox as bb
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
sns.set(style='darkgrid', context='notebook')  # Nicer plotting colors
bb.logger.setConsoleLevel('ERROR')             # Only show error log messages

anno = bb.io.load('anno_coco', '/content/annotations/instances_val2017.json')
anno.head()

In [ ]:
fields = ['image', 'class_label', 'id', 'x_top_left', 'y_top_left', 'width', 'height', "confidence"]
df = pd.DataFrame(results_cs_o, columns =fields)
df.to_pickle("res.pkl")
df.head()

In [ ]:
det = bb.io.load(
    'pandas',
    'res.pkl',
    class_label_map
    # We need to map the numbers from the JSON file to actual classnames
    #class_label_map=[lines.strip() for lines in open('coco.names')]
)

# The image labels are just numbers in the COCO json files, so we need to rename those as well
det.image = det.image.cat.rename_categories(lambda img: f'{img:012}')

# Finally, not all images might contain detections,
# and because the coco detection format does not contain a list of all images, we need to fix this ourselves
# Note that this is optional, but you will get a warning if you dont do this
det.image = det.image.cat.add_categories(set(anno.image.cat.categories) - set(det.image.cat.categories))

det.head()

In [ ]:
# Filter detections
det_filtered = det.sort_values('confidence').groupby('image').head(100)

# For each class
aps = []
for label in anno.class_label.unique():
    ac = anno[anno.class_label == label]
    dc = det_filtered[det_filtered.class_label == label]

    # Compute smoothed PR
    pr = bb.stat.pr(dc, ac, 0.5, smooth=True)

    # Compute interpolated Riemann
    aps.append(bb.stat.auc_interpolated(pr))

# Compute average mAP
mAP_50 = sum(aps) / len(aps)

mAP_50

In [ ]:
coco = bb.eval.COCO(det, anno)
coco.mAP